<a href="https://colab.research.google.com/github/worldwidekatie/GPT3_Synthetic/blob/master/GPT3_Synthetic_Data_Creaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPT3 Synthetic Training Data Research Project**
# Step One: Create Synthetic Data

In [1]:
!pip install openai

     |████████████████████████████████| 163kB 1.0MB/s 
  Created wheel for openai: filename=openai-0.2.4-cp36-none-any.whl size=170709 sha256=11ceade4ee9225bf23a260b06de23ebb713e3cbb86b6ce0814dcbc25140439af
  Stored in directory: /root/.cache/pip/wheels/74/96/c8/c6e170929c276b836613e1b9985343b501fe455e53d85e7d48
Successfully built openai


In [4]:
import openai
openai.api_key = ""

# Test #1, IMBD Movie Review Dataset

* Downloaded from [Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?select=IMDB+Dataset.csv)
* Split using a standard
 ```
 train, test = train_test_split(df, train_size=0.80, test_size=0.20, random_state=42)
 ```


## Load in the real training and test data

In [5]:
import pandas as pd

In [6]:
!wget 'https://github.com/worldwidekatie/GPT3_Synthetic/blob/master/imbd_train.zip?raw=true'

--2020-08-05 01:40:54--  https://github.com/worldwidekatie/GPT3_Synthetic/blob/master/imbd_train.zip?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/worldwidekatie/GPT3_Synthetic/raw/master/imbd_train.zip [following]
--2020-08-05 01:40:54--  https://github.com/worldwidekatie/GPT3_Synthetic/raw/master/imbd_train.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/worldwidekatie/GPT3_Synthetic/master/imbd_train.zip [following]
--2020-08-05 01:40:54--  https://raw.githubusercontent.com/worldwidekatie/GPT3_Synthetic/master/imbd_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443..

In [7]:
!unzip 'imbd_train.zip?raw=true'

Archive:  imbd_train.zip?raw=true
  inflating: imbd_train              


In [8]:
train = pd.read_csv('imbd_train')
train.shape

(40000, 2)

In [9]:
train = train.replace({'positive': 1, 'negative': 0})

In [10]:
train['sentiment'].value_counts(normalize=True)

0    0.500975
1    0.499025
Name: sentiment, dtype: float64

In [11]:
test = pd.read_csv('https://raw.githubusercontent.com/worldwidekatie/GPT3_Synthetic/master/imbd_test')
test.shape

(10000, 2)

In [12]:
test = test.replace({'positive': 1, 'negative': 0})

In [13]:
test['sentiment'].value_counts(normalize=True)

1    0.5039
0    0.4961
Name: sentiment, dtype: float64

# **Create GPT3-Generated Synthetic Dataset**

# 1%, No Shot
### First, take a random sample of 1% of the original training data to use as a seed.

In [ ]:
train_sample = train.sample(frac=.01, replace=False, random_state=79)
train_sample.shape

(400, 2)

### Split it into positive and negative

In [ ]:
train_sample_neg = train_sample[train_sample['sentiment']=='negative']
train_sample_neg.shape

(201, 2)

In [ ]:
train_sample_pos = train_sample[train_sample['sentiment']=='positive']
train_sample_pos.shape

(199, 2)

### Find the mean to use for max tokens in the data generation function

In [ ]:
num_tokens = []
for i in train['review']:
  tokens = []
  splt = i.split(" ")
  for i in splt:
    tokens.append(i)
  num_tokens.append(len(tokens))

num_tokens = pd.DataFrame(num_tokens)
num_tokens.describe()

,0
count,40000.000000
mean,230.995825
std,171.595625
min,4.000000
25%,126.000000
50%,173.000000
75%,280.000000
max,2470.000000


### Create the function to generate synthetic data from OpenAI

In [ ]:
def gen_data(df_column):
  data = []
  for i in df_column:
    response = openai.Completion.create(engine="ada", prompt=i, 
                                    max_tokens=231, temperature=.9, top_p=1, n=100, 
                                    stream=False)
    for i in response['choices']:
      data.append(i['text'])
  return data

### Generate synthetic negative review data.

In [ ]:
synth_data_neg = gen_data(train_sample_neg['review'])
len(synth_data_neg)

20100

In [ ]:
synth_train_neg = pd.DataFrame(synth_data_neg, columns=['review'])

In [ ]:
synth_train_neg['sentiment'] = 'negative'
synth_train_neg

,review,sentiment
0,You just might be looking at a proper race to...,negative
1,\n\n6.0 out of 5 stars Plus one - an ingenious...,negative
2,Be your usual professional and watch the othe...,negative
3,<br /><br />So sit back and enjoy a man who w...,negative
4,,negative
...,...,...
20095,,negative
20096,The book is free - buy it. A few of the review...,negative
20097,,negative
20098,KiA,negative


In [ ]:
synth_train_neg.to_csv('gpt3_imbd_neg.csv')

In [ ]:
from google.colab import files
files.download('gpt3_imbd_neg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate synthetic positive review data.

In [ ]:
synth_data_pos = gen_data(train_sample_pos['review'])
len(synth_data_pos)

19900

In [ ]:
synth_train_pos = pd.DataFrame(synth_data_pos, columns=['review'])

In [ ]:
synth_train_pos['sentiment'] = 'positive'
synth_train_pos

,review,sentiment
0,,positive
1,The Man who Shot Martinsons is a very well mad...,positive
2,But Cordova's also one of the few actors worki...,positive
3,"The music in the film is pretty amazing, espec...",positive
4,It's not an easy film to enjoy and I wouldn't ...,positive
...,...,...
19895,(And that's a serious observation - not exact...,positive
19896,It's much more hilarious than if you just wat...,positive
19897,\n\nRated 5 out of 5 stars (472) Rank: #1 Revi...,positive
19898,"Throw Momma from the Train is a fun comedy, b...",positive


In [ ]:
synth_train_pos.to_csv('gpt3_imbd_pos.csv')

In [ ]:
from google.colab import files
files.download('gpt3_imbd_pos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Concatonate them for one large training dataset.

In [ ]:
gpt_train = pd.concat([synth_train_pos, synth_train_neg])

In [ ]:
gpt_train.to_csv('gpt3_imbd_train1.csv', index=False)

In [ ]:
from google.colab import files
files.download('gpt3_imbd_train1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
gpt_train.shape

(40000, 2)

In [ ]:
gpt_train['sentiment'].value_counts(normalize=True)

negative    0.5025
positive    0.4975
Name: sentiment, dtype: float64

# 1%, Two Shots
### First, take a random sample of 1% of the original training data to use as a seed.

In [14]:
train_sample = train.sample(frac=.01, replace=False, random_state=79)
train_sample.shape

(400, 2)

### Split it into positive and negative

In [92]:
train_sample_neg = train_sample[train_sample['sentiment']==0]
train_sample_neg.shape

(201, 2)

In [93]:
train_sample_pos = train_sample[train_sample['sentiment']==1]
train_sample_pos.shape

(199, 2)

### Find the mean to use for max tokens in the data generation function

In [ ]:
num_tokens = []
for i in train['review']:
  tokens = []
  splt = i.split(" ")
  for i in splt:
    tokens.append(i)
  num_tokens.append(len(tokens))

num_tokens = pd.DataFrame(num_tokens)
num_tokens.describe()

,0
count,40000.000000
mean,230.995825
std,171.595625
min,4.000000
25%,126.000000
50%,173.000000
75%,280.000000
max,2470.000000


### Create the function to generate synthetic data from OpenAI

In [68]:
import random
import numpy as np

In [26]:
test_df = train_sample_neg['review'].head()

In [88]:
def gen_data(df_column):
  data = []
  var = list(df_column)
  for i in var:
    x = pd.DataFrame([var[random.randrange(len(var))], i])
    
    response = openai.Completion.create(engine="ada", prompt=list(x), 
                                    max_tokens=231, temperature=.9, top_p=1, n=100, 
                                    stream=False)
    for i in response['choices']:
      data.append(i['text'])
  return data

### Generate synthetic negative review data.

In [94]:
synth_data_neg = gen_data(train_sample_neg['review'])
len(synth_data_neg)

20100

In [95]:
synth_train_neg = pd.DataFrame(synth_data_neg, columns=['review'])

In [96]:
synth_train_neg['sentiment'] = 0
synth_train_neg

,review,sentiment
0,\n\n\n\nFeatured Image from Davids Store\n\nFo...,0
1,\n\nR = They all suck!\n\nA = Adventure!!\n\n\...,0
2,Thank you Tavis Brothers! Have a great day! 😊...,0
3,2f[td][td][td][td][td][td][td][td][td][td][td]...,0
4,\n\n8/15/14\n\nYeeze Blig!\n\n\n\n\n\nThis onl...,0
...,...,...
20095,\n\nshpw – I was wondering whether a stylized ...,0
20096,""" ""How often do you have to send me e-mails l...",0
20097,\n\nGym ZZ\n\n\n\n“Zz” | Final | 12/20/17 @ 12...,0
20098,In the end the death of of child child won. 2...,0


In [99]:
synth_train_neg.to_csv('gpt3_imbd_neg1.csv', index=False)

In [100]:
from google.colab import files
files.download('gpt3_imbd_neg1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate synthetic positive review data.

In [101]:
synth_data_pos = gen_data(train_sample_pos['review'])
len(synth_data_pos)

19900

In [102]:
synth_train_pos = pd.DataFrame(synth_data_pos, columns=['review'])

In [103]:
synth_train_pos['sentiment'] = 1
synth_train_pos

,review,sentiment
0,Buy a hard shell for $30 each.\n\n\n\nFinal Q...,1
1,John Berard's Flea Key Sesamum Indicum To Say...,1
2,(parallel)\n\nIn a device description of COM ...,1
3,”\n\n“But you hate Tom!”\n\n“Do I love him or ...,1
4,"Hilarity of ""happy people!"" Ahem. How is it e...",1
...,...,...
19895,"I'll settle for not finding out. ""\n\nWe didn...",1
19896,They hate you the way you are. And I'll tell ...,1
19897,"\n\nTo use this interface, all you need is a C...",1
19898,"This is more an outline, not a visual guidebo...",1


In [104]:
synth_train_pos.to_csv('gpt3_imbd_pos1.csv')

In [105]:
from google.colab import files
files.download('gpt3_imbd_pos1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Concatonate them for one large training dataset.

In [106]:
gpt_train = pd.concat([synth_train_pos, synth_train_neg])

In [107]:
gpt_train.to_csv('gpt3_imbd_train1_1.csv', index=False)

In [108]:
from google.colab import files
files.download('gpt3_imbd_train1_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [109]:
gpt_train.shape

(40000, 2)

In [110]:
gpt_train['sentiment'].value_counts(normalize=True)

0    0.5025
1    0.4975
Name: sentiment, dtype: float64